# PrepareNeo4jBulkImport
This notebook uses metadata to prepare CSV Files for Neo4j Bulk Data Import

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import os
from os import walk
from os.path import join
import pandas as pd
from utils import create_node_headers, create_relationship_headers, get_node_data_headers, get_relationship_data_headers, create_meta_node, create_meta_relationship

In [2]:
# reload modules before executing user code
%load_ext autoreload
%autoreload 2

In [3]:
# configure pandas dataframe
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns
pd.set_option('display.max_colwidth', None)

## Specify working directories

If NEO4J_HOME directory is not set, this notebook creates the example_results/import directory as a proxy for a Neo4j import directory.

In [4]:
NEO4J_HOME = os.getenv('NEO4J_HOME', default='../example_results')

Location of Neo4j import directory

In [5]:
NEO4J_IMPORT = os.path.join(NEO4J_HOME, 'import')

Create the example_results/import directory for testing the example provided in this repo.

In [6]:
if NEO4J_HOME == '../example_results':
    os.makedirs(os.path.join(NEO4J_IMPORT), exist_ok=True)

Locations of metadata files (Defaults are files for testing this notebook)

In [7]:
NODE_METADATA = os.getenv('NODE_METADATA', default='../example_metadata/nodes/')
RELATIONSHIP_METADATA = os.getenv('RELATIONSHIP_METADATA', default='../example_metadata/relationships/')

Locations of data files (Defaults are files for testing this notebook)

In [8]:
NODE_DATA = os.getenv('NODE_DATA', default='../example_data/nodes/')
RELATIONSHIP_DATA = os.getenv('RELATIONSHIP_DATA', default='../example_data/relationships/')

## Create headers from metadata files

Create the node name, the expected node file header, and the Neo4j header for bulk import from the metadata files.

In [9]:
dirpath, _, filenames = next(walk(NODE_METADATA))
node_headers = [create_node_headers(dirpath, filename) for filename in filenames]

In [10]:
node_metadata = pd.DataFrame(node_headers)

In [11]:
node_metadata

,node,metadataHeader,importHeader,metadataPath
0,Patient,"id,firstName,lastName,age,sex,smoker","id:ID(Patient-ID),firstName:string,lastName:string,age:int,sex:string,smoker:boolean",../example_metadata/nodes/Patient.csv
1,State,"id,name,synonyms,population,location","id:ID(State-ID),name:string,synonyms:string[],population:int,location:point{crs:WGS-84}",../example_metadata/nodes/State.csv
2,City,"id,name,synonyms,population,location","id:ID(City-ID),name:string,synonyms:string[],population:int,location:point{crs:WGS-84}",../example_metadata/nodes/City.csv
3,Symptom,"id,name","id:ID(Symptom-ID),name:string",../example_metadata/nodes/Symptom.csv
4,Disease,"id,name","id:ID(Disease-ID),name:string",../example_metadata/nodes/Disease.csv


Create the relationship name, the expected relatinonship file header, and the Neo4j header for bulk import from the metadata files.

In [12]:
dirpath, _, filenames = next(walk(RELATIONSHIP_METADATA))
relationship_headers = [create_relationship_headers(dirpath, filename) for filename in filenames]

In [13]:
relationship_metadata = pd.DataFrame(relationship_headers)

In [14]:
relationship_metadata

,relationship,metadataHeader,importHeader,source,target,metadataPath
0,LOCATED_IN,"from,to",":START_ID(City-ID),:END_ID(State-ID)",City,State,../example_metadata/relationships/City-LOCATED_IN-State.csv
1,DIAGNOSED_WITH,"from,to,diagnosisDate",":START_ID(Patient-ID),:END_ID(Disease-ID),diagnosisDate:date",Patient,Disease,../example_metadata/relationships/Patient-DIAGNOSED_WITH-Disease.csv
2,LIVES_IN,"from,to",":START_ID(Patient-ID),:END_ID(City-ID)",Patient,City,../example_metadata/relationships/Patient-LIVES_IN-City.csv
3,SHOWS,"from,to,startDate",":START_ID(Patient-ID),:END_ID(Symptom-ID),startDate:date",Patient,Symptom,../example_metadata/relationships/Patient-SHOWS-Symptom.csv
4,PRESENTS,"from,to",":START_ID(Disease-ID),:END_ID(Symptom-ID)",Disease,Symptom,../example_metadata/relationships/Disease-PRESENTS-Symptom.csv


## Get headers from data files

Node data files

In [15]:
dirpath, _, filenames = next(walk(NODE_DATA))
csv_files = filter(lambda name: name.endswith('.csv'), filenames)
data_headers = [get_node_data_headers(dirpath, filename) for filename in csv_files]

Node Data ../example_data/nodes/ State.csv
Node Data ../example_data/nodes/ City.csv
Node Data ../example_data/nodes/ Symptom.csv
Node Data ../example_data/nodes/ Patient_2021.csv
Node Data ../example_data/nodes/ Disease.csv
Node Data ../example_data/nodes/ Patient_2020.csv


In [16]:
node_data = pd.DataFrame(data_headers)

In [17]:
node_data

,node,dataHeader,dataPath
0,State,"id,name,synonyms,population,location",../example_data/nodes/State.csv
1,City,"id,name,synonyms,population,location",../example_data/nodes/City.csv
2,Symptom,"id,name",../example_data/nodes/Symptom.csv
3,Patient,"id,firstName,lastName,age,sex,smoker",../example_data/nodes/Patient_2021.csv
4,Disease,"id,name",../example_data/nodes/Disease.csv
5,Patient,"id,firstName,lastName,age,sex,smoker",../example_data/nodes/Patient_2020.csv


Relationship data files

In [18]:
dirpath, _, filenames = next(walk(RELATIONSHIP_DATA))
csv_files = filter(lambda name: name.endswith('.csv'), filenames)
data_headers = [get_relationship_data_headers(dirpath, filename) for filename in csv_files]

Relationship Data ../example_data/relationships/ City-LOCATED_IN-State.csv
Relationship Data ../example_data/relationships/ Patient-DIAGNOSED_WITH-Disease_2021.csv
Relationship Data ../example_data/relationships/ Patient-DIAGNOSED_WITH-Disease_2020.csv
Relationship Data ../example_data/relationships/ Patient-LIVES_IN-City.csv
Relationship Data ../example_data/relationships/ Patient-SHOWS-Symptom.csv
Relationship Data ../example_data/relationships/ Disease-PRESENTS-Symptom.csv


In [19]:
relationship_data = pd.DataFrame(data_headers)

In [20]:
relationship_data

,relationship,dataHeader,dataPath
0,LOCATED_IN,"from,to",../example_data/relationships/City-LOCATED_IN-State.csv
1,DIAGNOSED_WITH,"from,to,diagnosisDate",../example_data/relationships/Patient-DIAGNOSED_WITH-Disease_2021.csv
2,DIAGNOSED_WITH,"from,to,diagnosisDate",../example_data/relationships/Patient-DIAGNOSED_WITH-Disease_2020.csv
3,LIVES_IN,"from,to",../example_data/relationships/Patient-LIVES_IN-City.csv
4,SHOWS,"from,to,startDate",../example_data/relationships/Patient-SHOWS-Symptom.csv
5,PRESENTS,"from,to",../example_data/relationships/Disease-PRESENTS-Symptom.csv


## Merge metadata with data

In [21]:
matched_nodes = node_data.merge(node_metadata, on='node', how='outer')
matched_nodes['match'] = matched_nodes['dataHeader'] == matched_nodes['metadataHeader']
matched_nodes.fillna('', inplace=True)

In [22]:
matched_nodes

,node,dataHeader,dataPath,metadataHeader,importHeader,metadataPath,match
0,State,"id,name,synonyms,population,location",../example_data/nodes/State.csv,"id,name,synonyms,population,location","id:ID(State-ID),name:string,synonyms:string[],population:int,location:point{crs:WGS-84}",../example_metadata/nodes/State.csv,True
1,City,"id,name,synonyms,population,location",../example_data/nodes/City.csv,"id,name,synonyms,population,location","id:ID(City-ID),name:string,synonyms:string[],population:int,location:point{crs:WGS-84}",../example_metadata/nodes/City.csv,True
2,Symptom,"id,name",../example_data/nodes/Symptom.csv,"id,name","id:ID(Symptom-ID),name:string",../example_metadata/nodes/Symptom.csv,True
3,Patient,"id,firstName,lastName,age,sex,smoker",../example_data/nodes/Patient_2021.csv,"id,firstName,lastName,age,sex,smoker","id:ID(Patient-ID),firstName:string,lastName:string,age:int,sex:string,smoker:boolean",../example_metadata/nodes/Patient.csv,True
4,Patient,"id,firstName,lastName,age,sex,smoker",../example_data/nodes/Patient_2020.csv,"id,firstName,lastName,age,sex,smoker","id:ID(Patient-ID),firstName:string,lastName:string,age:int,sex:string,smoker:boolean",../example_metadata/nodes/Patient.csv,True
5,Disease,"id,name",../example_data/nodes/Disease.csv,"id,name","id:ID(Disease-ID),name:string",../example_metadata/nodes/Disease.csv,True


In [23]:
mismatched_nodes = matched_nodes[(matched_nodes['match'] == False) & (matched_nodes['dataPath'] != '')]
mismatched_nodes

,node,dataHeader,dataPath,metadataHeader,importHeader,metadataPath,match


In [24]:
if mismatched_nodes.shape[0] > 0:
    print('The following node data files do not match the metadata specification:')
    mismatched_nodes.to_csv(os.path.join(NEO4J_IMPORT, 'mismatches_n.csv'), index=False)
    
mismatched_nodes

,node,dataHeader,dataPath,metadataHeader,importHeader,metadataPath,match


In [25]:
matched_relationships = relationship_data.merge(relationship_metadata, on='relationship', how='outer')
matched_relationships['match'] = matched_relationships['dataHeader'] == matched_relationships['metadataHeader']
matched_relationships.fillna('', inplace=True)
# pr
matched_relationships['fullRelationship'] = matched_relationships['source'] + '-' + matched_relationships['relationship'] + '-' + matched_relationships['target']
matched_relationships

,relationship,dataHeader,dataPath,metadataHeader,importHeader,source,target,metadataPath,match,fullRelationship
0,LOCATED_IN,"from,to",../example_data/relationships/City-LOCATED_IN-State.csv,"from,to",":START_ID(City-ID),:END_ID(State-ID)",City,State,../example_metadata/relationships/City-LOCATED_IN-State.csv,True,City-LOCATED_IN-State
1,DIAGNOSED_WITH,"from,to,diagnosisDate",../example_data/relationships/Patient-DIAGNOSED_WITH-Disease_2021.csv,"from,to,diagnosisDate",":START_ID(Patient-ID),:END_ID(Disease-ID),diagnosisDate:date",Patient,Disease,../example_metadata/relationships/Patient-DIAGNOSED_WITH-Disease.csv,True,Patient-DIAGNOSED_WITH-Disease
2,DIAGNOSED_WITH,"from,to,diagnosisDate",../example_data/relationships/Patient-DIAGNOSED_WITH-Disease_2020.csv,"from,to,diagnosisDate",":START_ID(Patient-ID),:END_ID(Disease-ID),diagnosisDate:date",Patient,Disease,../example_metadata/relationships/Patient-DIAGNOSED_WITH-Disease.csv,True,Patient-DIAGNOSED_WITH-Disease
3,LIVES_IN,"from,to",../example_data/relationships/Patient-LIVES_IN-City.csv,"from,to",":START_ID(Patient-ID),:END_ID(City-ID)",Patient,City,../example_metadata/relationships/Patient-LIVES_IN-City.csv,True,Patient-LIVES_IN-City
4,SHOWS,"from,to,startDate",../example_data/relationships/Patient-SHOWS-Symptom.csv,"from,to,startDate",":START_ID(Patient-ID),:END_ID(Symptom-ID),startDate:date",Patient,Symptom,../example_metadata/relationships/Patient-SHOWS-Symptom.csv,True,Patient-SHOWS-Symptom
5,PRESENTS,"from,to",../example_data/relationships/Disease-PRESENTS-Symptom.csv,"from,to",":START_ID(Disease-ID),:END_ID(Symptom-ID)",Disease,Symptom,../example_metadata/relationships/Disease-PRESENTS-Symptom.csv,True,Disease-PRESENTS-Symptom


In [26]:
mismatched_relationships = matched_relationships[(matched_relationships['match'] == False) & (matched_relationships['dataPath'] != '')]
if mismatched_relationships.shape[0] > 0:
    print('The following relationship data files do not match the metadata specification:')
    mismatched_relationships.to_csv(os.path.join(NEO4J_IMPORT, 'mismatches_r.csv'), index=False)
    
mismatched_relationships

,relationship,dataHeader,dataPath,metadataHeader,importHeader,source,target,metadataPath,match,fullRelationship


## Write Neo4j header files for bulk import

In [27]:
def save_node_header(name, import_header, NEO4J_IMPORT):
    df = pd.DataFrame([], columns=import_header.split(','))
    df.to_csv(os.path.join(NEO4J_IMPORT, 'header_' + name + '_n.csv'), index=False)

Write node header files

In [28]:
matched_nodes.query('match == True', inplace=True)

In [29]:
out = matched_nodes.apply(lambda row: save_node_header(row['node'], row['importHeader'], NEO4J_IMPORT), axis=1)

Write relationship header files

In [30]:
def save_relationship_header(name, import_header, NEO4J_IMPORT):
    df = pd.DataFrame([], columns=import_header.split(','))
    df.to_csv(os.path.join(NEO4J_IMPORT, 'header_' + name + '_r.csv'), index=False)

In [31]:
matched_relationships.query('match == True', inplace=True)

In [32]:
matched_relationships

,relationship,dataHeader,dataPath,metadataHeader,importHeader,source,target,metadataPath,match,fullRelationship
0,LOCATED_IN,"from,to",../example_data/relationships/City-LOCATED_IN-State.csv,"from,to",":START_ID(City-ID),:END_ID(State-ID)",City,State,../example_metadata/relationships/City-LOCATED_IN-State.csv,True,City-LOCATED_IN-State
1,DIAGNOSED_WITH,"from,to,diagnosisDate",../example_data/relationships/Patient-DIAGNOSED_WITH-Disease_2021.csv,"from,to,diagnosisDate",":START_ID(Patient-ID),:END_ID(Disease-ID),diagnosisDate:date",Patient,Disease,../example_metadata/relationships/Patient-DIAGNOSED_WITH-Disease.csv,True,Patient-DIAGNOSED_WITH-Disease
2,DIAGNOSED_WITH,"from,to,diagnosisDate",../example_data/relationships/Patient-DIAGNOSED_WITH-Disease_2020.csv,"from,to,diagnosisDate",":START_ID(Patient-ID),:END_ID(Disease-ID),diagnosisDate:date",Patient,Disease,../example_metadata/relationships/Patient-DIAGNOSED_WITH-Disease.csv,True,Patient-DIAGNOSED_WITH-Disease
3,LIVES_IN,"from,to",../example_data/relationships/Patient-LIVES_IN-City.csv,"from,to",":START_ID(Patient-ID),:END_ID(City-ID)",Patient,City,../example_metadata/relationships/Patient-LIVES_IN-City.csv,True,Patient-LIVES_IN-City
4,SHOWS,"from,to,startDate",../example_data/relationships/Patient-SHOWS-Symptom.csv,"from,to,startDate",":START_ID(Patient-ID),:END_ID(Symptom-ID),startDate:date",Patient,Symptom,../example_metadata/relationships/Patient-SHOWS-Symptom.csv,True,Patient-SHOWS-Symptom
5,PRESENTS,"from,to",../example_data/relationships/Disease-PRESENTS-Symptom.csv,"from,to",":START_ID(Disease-ID),:END_ID(Symptom-ID)",Disease,Symptom,../example_metadata/relationships/Disease-PRESENTS-Symptom.csv,True,Disease-PRESENTS-Symptom


In [33]:
# out = matched_relationships.apply(lambda row: save_relationship_header(row['relationship'], row['importHeader'], NEO4J_IMPORT), axis=1)
out = matched_relationships.apply(lambda row: save_relationship_header(row['fullRelationship'], row['importHeader'], NEO4J_IMPORT), axis=1)

## Create MetaNode and MetaRelationship files

Compile a dictionary of all node properties

In [34]:
property_dir = {}
for header in matched_nodes['metadataHeader']:
    for property in header.split(','):
        property_dir[property] = ''

Create dataframe with MetaNode data

In [35]:
node_list = []
for node, filepath in matched_nodes[['node','metadataPath']].itertuples(index=False):
    node_list.append(create_meta_node(node, property_dir, filepath))
    
meta_nodes = pd.DataFrame(node_list)
meta_nodes.drop_duplicates(inplace=True)
meta_nodes.to_csv(os.path.join(NEO4J_IMPORT, 'MetaNode_n.csv'), index=False)

In [36]:
meta_nodes

,id,name,synonyms,population,location,firstName,lastName,age,sex,smoker,nodeName:ID(MetaNode-ID)
0,Geonames.org id for location (string),Name of state (string),Alternate names of state (string[]),Population (int),Latitude and longitude in WGS-84 format (point{crs:WGS-84}),,,,,,State
1,Geonames.org id for location (string),Name of city (string),Alternate names of city (string[]),Population (int),Latitude and longitude in WGS-84 format (point{crs:WGS-84}),,,,,,City
2,Symptom id from Symptom Ontology (string),Name of symptom (string),,,,,,,,,Symptom
3,Unique patient id (string),,,,,First name (string),Last name (string),Age (int),Biological sex (string),Patient is a smoker (boolean),Patient
5,Disease id from Human Disease Ontology (string),Name of disease from Human Disease Ontology (string),,,,,,,,,Disease


Compile a dictionary of all relationship properties

In [37]:
property_dir = {}
for header in matched_relationships['metadataHeader']:
    for property in header.split(','):
        property_dir[property] = ''

Create dataframe with MetaRelationship data

In [38]:
relationship_list = []
for relationship, source, target, filepath in matched_relationships[['relationship', 'source', 'target', 'metadataPath']].itertuples(index=False):
    relationship_list.append(create_meta_relationship(relationship, source, target, property_dir, filepath))
    
meta_relationships = pd.DataFrame(relationship_list)
meta_relationships.drop_duplicates(inplace=True)
meta_relationships.to_csv(os.path.join(NEO4J_IMPORT, 'MetaRelationship_r.csv'), index=False)

In [39]:
meta_relationships

,from,to,diagnosisDate,startDate,relationshipName,source:START_ID(MetaNode-ID),target:END_ID(MetaNode-ID)
0,Id of source node (string),Id of target node (string),,,LOCATED_IN,City,State
1,Id of source node (string),Id of target node (string),Date of diagnosis (date),,DIAGNOSED_WITH,Patient,Disease
3,Id of source node (string),Id of target node (string),,,LIVES_IN,Patient,City
4,Id of source node (string),Id of target node (string),,Date when symptom started (date),SHOWS,Patient,Symptom
5,Id of source node (string),Id of target node (string),,,PRESENTS,Disease,Symptom


## Create Neo4j bulk upload command line arguments
See: https://neo4j.com/docs/operations-manual/current/tools/neo4j-admin/neo4j-admin-import/

In [40]:
args = ''
for node in matched_nodes['node'].unique():
    args += f' --nodes={node}=header_{node}_n.csv,{node}.*_n.csv'

In [41]:
args += ' --nodes=MetaNode=MetaNode_n.csv'

In [42]:
rel_data = matched_relationships[['relationship', 'fullRelationship']].copy()
rel_data.drop_duplicates(inplace=True)
for relationship, fullRelationship in rel_data.itertuples(index=False):
    # args += f' --relationships={relationship}=header_{fullRelationship}_r.csv,{fullRelationship}_.*_r.csv'
    args += f' --relationships={relationship}=header_{fullRelationship}_r.csv,{fullRelationship}.*_r.csv'

In [43]:
args += ' --relationships=MetaRelationship=MetaRelationship_r.csv'

In [44]:
file = open(os.path.join(NEO4J_IMPORT, 'args.txt'), 'w')
file.write(args)
file.close() 

In [45]:
print(args)

 --nodes=State=header_State_n.csv,State.*_n.csv --nodes=City=header_City_n.csv,City.*_n.csv --nodes=Symptom=header_Symptom_n.csv,Symptom.*_n.csv --nodes=Patient=header_Patient_n.csv,Patient.*_n.csv --nodes=Disease=header_Disease_n.csv,Disease.*_n.csv --nodes=MetaNode=MetaNode_n.csv --relationships=LOCATED_IN=header_City-LOCATED_IN-State_r.csv,City-LOCATED_IN-State.*_r.csv --relationships=DIAGNOSED_WITH=header_Patient-DIAGNOSED_WITH-Disease_r.csv,Patient-DIAGNOSED_WITH-Disease.*_r.csv --relationships=LIVES_IN=header_Patient-LIVES_IN-City_r.csv,Patient-LIVES_IN-City.*_r.csv --relationships=SHOWS=header_Patient-SHOWS-Symptom_r.csv,Patient-SHOWS-Symptom.*_r.csv --relationships=PRESENTS=header_Disease-PRESENTS-Symptom_r.csv,Disease-PRESENTS-Symptom.*_r.csv --relationships=MetaRelationship=MetaRelationship_r.csv
